In [ ]:
%%capture

from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

%run s05_ai_engine.ipynb

# Dirty hack to import types from other file for static analysis tools
try:
    from . import ScoredMove, IterativeAlphaBetaCached, HumanEngine, playGame
    import chess
    import random
except ImportError:
    pass

# Enhanced AI Engine

```{warning}
This section is work in progress and contains only unfinished code snippets
``` 

## MDTF Search Algorithm

In [ ]:
import math


class IterativeMtdf(IterativeAlphaBetaCached):
    """Chess engine looking a fixed number of moves ahead using the alpha beta pruning algorithm"""

    def _mtdf(self, board: chess.Board, score: int, depth: int) -> int:
        upperbound = math.inf
        lowerbound = -math.inf

        while lowerbound < upperbound:
            beta = score + 1 if score == lowerbound else score
            score = self._value(board, depth, beta - 1, beta)
            if score < beta:
                upperbound = score
            else:
                lowerbound = score

        return score

    def _evaluate_move(
        self, board: chess.Board, scoredMove: ScoredMove, depth: int
    ):
        board.push(scoredMove.move)
        score = self._mtdf(board, scoredMove.score, depth)
        score *= self.PLAYER_MULTIPLIER[board.turn]
        board.pop()
        return ScoredMove(score=score, move=scoredMove.move)

    def _evaluate_moves(self, board: chess.Board):
        print(f"Max depth: {self.max_look_ahead_depth}")

        self.cache.clear()

        scoredMoves = [
            ScoredMove(score=0, move=move) for move in board.legal_moves
        ]

        for depth in range(self.max_look_ahead_depth + 1):
            scoredMoves = [
                self._evaluate_move(board, move, depth) for move in scoredMoves
            ]

            print(f"Depth {depth}")
            # print(f"result: {scoredMoves}\n")

        return scoredMoves

In [ ]:
import MinMaxTree

random.seed(42)

engine = IterativeMtdf(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
%time result_depth3_IterativeMtdf = engine.analyse(middlegame_board)

assert result_depth3_IterativeMtdf == result_depth3_iterativeAlphaBetaCached

In [ ]:
# Play against the engine

# random.seed(42)
# board = chess.Board()

# engine = IterativeMtdf()
# engine.MAX_LOOK_AHEAD_DEPTH = 3

# playGame(board, engine, HumanEngine(), displayBoard=True)
# IPython.display.display(board)
# print(board.outcome())

## Simplified Evaluation Function

In [ ]:
class IterativeMtdfSimplifiedEvaluation(IterativeMtdf):
    pass


PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

IterativeMtdfSimplifiedEvaluation.PIECE_VALUES = PIECE_VALUES

IterativeMtdfSimplifiedEvaluation.VALUE_CHECKMATE = 30.000
IterativeMtdfSimplifiedEvaluation.VALUE_DRAW = 0

PIECE_SQUARE_VALUES = {
    chess.PAWN: [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [50, 50, 50, 50, 50, 50, 50, 50],
        [10, 10, 20, 30, 30, 20, 10, 10],
        [ 5,  5, 10, 25, 25, 10,  5,  5],
        [ 0,  0,  0, 20, 20,  0,  0,  0],
        [ 5, -5,-10,  0,  0,-10, -5,  5],
        [ 5, 10, 10,-20,-20, 10, 10,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0]
    ],
    chess.KNIGHT: [
        [-50,-40,-30,-30,-30,-30,-40,-50],
        [-40,-20,  0,  0,  0,  0,-20,-40],
        [-30,  0, 10, 15, 15, 10,  0,-30],
        [-30,  5, 15, 20, 20, 15,  5,-30],
        [-30,  0, 15, 20, 20, 15,  0,-30],
        [-30,  5, 10, 15, 15, 10,  5,-30],
        [-40,-20,  0,  5,  5,  0,-20,-40],
        [-50,-40,-30,-30,-30,-30,-40,-50]
    ],
    chess.BISHOP: [
        [-20,-10,-10,-10,-10,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5, 10, 10,  5,  0,-10],
        [-10,  5,  5, 10, 10,  5,  5,-10],
        [-10,  0, 10, 10, 10, 10,  0,-10],
        [-10, 10, 10, 10, 10, 10, 10,-10],
        [-10,  5,  0,  0,  0,  0,  5,-10],
        [-20,-10,-10,-10,-10,-10,-10,-20]
    ],
    chess.ROOK: [
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 5, 10, 10, 10, 10, 10, 10,  5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [-5,  0,  0,  0,  0,  0,  0, -5],
        [ 0,  0,  0,  5,  5,  0,  0,  0]
    ],
    chess.QUEEN: [
        [-20,-10,-10, -5, -5,-10,-10,-20],
        [-10,  0,  0,  0,  0,  0,  0,-10],
        [-10,  0,  5,  5,  5,  5,  0,-10],
        [ -5,  0,  5,  5,  5,  5,  0, -5],
        [  0,  0,  5,  5,  5,  5,  0, -5],
        [-10,  5,  5,  5,  5,  5,  0,-10],
        [-10,  0,  5,  0,  0,  0,  0,-10],
        [-20,-10,-10, -5, -5,-10,-10,-20]
    ],
    # TODO: Add support for endgame king square table
    chess.KING: [
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-30,-40,-40,-50,-50,-40,-40,-30],
        [-20,-30,-30,-40,-40,-30,-30,-20],
        [-10,-20,-20,-20,-20,-20,-20,-10],
        [ 20, 20,  0,  0,  0,  0, 20, 20],
        [ 20, 30, 10,  0,  0, 10, 30, 20],
    ]
} # yapf: disable
IterativeMtdfSimplifiedEvaluation.PIECE_SQUARE_VALUES = PIECE_SQUARE_VALUES


def _absolute_heuristic(self, board: chess.Board) -> int:
    """Calculate the value of all pieces on the board"""
    score = 0

    for square in chess.scan_reversed(board.occupied_co[board.turn]):
        piece_type = board.piece_type_at(square)
        file = chess.square_file(square)
        rank = chess.square_rank(square)
        score += self.PIECE_VALUES[piece_type]
        score += self.PIECE_SQUARE_VALUES[piece_type][file][rank]

    for square in chess.scan_reversed(board.occupied_co[not board.turn]):
        piece_type = board.piece_type_at(square)
        file = chess.square_file(square)
        rank = chess.square_rank(square)
        score -= self.PIECE_VALUES[piece_type]
        score -= self.PIECE_SQUARE_VALUES[piece_type][file][rank]

    return score


IterativeMtdfSimplifiedEvaluation._absolute_heuristic = _absolute_heuristic

In [ ]:
random.seed(42)

engine = IterativeMtdfSimplifiedEvaluation(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)

IPython.display.display(middlegame_board)
print(engine._absolute_heuristic(middlegame_board))

In [ ]:
engine = IterativeMtdf(max_look_ahead_depth=3)
%timeit engine._absolute_heuristic(middlegame_board)
engine = IterativeMtdfSimplifiedEvaluation(max_look_ahead_depth=3)
%timeit engine._absolute_heuristic(middlegame_board)

In [ ]:
random.seed(42)

engine = IterativeMtdfSimplifiedEvaluation(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
# %time engine.analyse(middlegame_board)

## Quience Search